In [69]:
# importing libraries

import pandas as pd
import numpy as np
import re

In [70]:
# importing data

df = pd.read_csv(r'C:\Users\CharlesYi\Jupyter Notebook\Alteryx Challenges\Challenge 11_Identify Logical Groups\data.csv')

df.head()

,Field_6
0,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED..."
1,"ON MAY 10 2015, THE REPORTER CONTACTED THE COM..."
2,DATE OF SUBMISSION 06/10/2011. DEVICE EVALUATI...
3,DATE OF SUBMISSION 08/21/2017 - DEVICE EVALUAT...
4,IT WAS REPORTED THAT THE DEVICE ALARMED AND IN...


In [71]:
lookup = pd.read_csv(r'C:\Users\CharlesYi\Jupyter Notebook\Alteryx Challenges\Challenge 11_Identify Logical Groups\lookup.csv')

lookup.head()

,Search,Bucket
0,beep,Tones
1,screen,Screen
2,trigger,Trigger


In [72]:
# appending lookup to data

df_merged = pd.merge(df, lookup, how = 'cross')

df_merged.head()

,Field_6,Search,Bucket
0,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",beep,Tones
1,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",screen,Screen
2,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",trigger,Trigger
3,"ON MAY 10 2015, THE REPORTER CONTACTED THE COM...",beep,Tones
4,"ON MAY 10 2015, THE REPORTER CONTACTED THE COM...",screen,Screen


In [73]:
# changing field name

col_rename = {'Field_6' : 'Text'}

df_merged.rename(columns = col_rename, inplace = True)

df_merged.head()

,Text,Search,Bucket
0,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",beep,Tones
1,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",screen,Screen
2,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",trigger,Trigger
3,"ON MAY 10 2015, THE REPORTER CONTACTED THE COM...",beep,Tones
4,"ON MAY 10 2015, THE REPORTER CONTACTED THE COM...",screen,Screen


In [74]:
# checking if search term is in text

df_merged['Occurence'] = df_merged.apply(lambda x: 1 if x['Text'].lower().find(x['Search'].lower()) != -1 else 0, axis = 1)

df_merged.head()

,Text,Search,Bucket,Occurence
0,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",beep,Tones,0
1,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",screen,Screen,1
2,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",trigger,Trigger,0
3,"ON MAY 10 2015, THE REPORTER CONTACTED THE COM...",beep,Tones,0
4,"ON MAY 10 2015, THE REPORTER CONTACTED THE COM...",screen,Screen,1


In [75]:
# aggregating occurence by text and bucket

df_agg = df_merged.groupby(['Text', 'Bucket']).agg(sum_occ = ('Occurence' , 'sum'))

# resetting index

df_agg.reset_index(inplace = True)

df_agg.head()

,Text,Bucket,sum_occ
0,CUSTOMER STATED THAT THE DEVICE ALARMED BUTTON...,Screen,1
1,CUSTOMER STATED THAT THE DEVICE ALARMED BUTTON...,Tones,0
2,CUSTOMER STATED THAT THE DEVICE ALARMED BUTTON...,Trigger,0
3,DATE OF SUBMISSION 06/10/2011. DEVICE EVALUATI...,Screen,1
4,DATE OF SUBMISSION 06/10/2011. DEVICE EVALUATI...,Tones,0


In [76]:
# crosstabbing bucket

df_pivot = df_agg.pivot_table(index = 'Text', columns = 'Bucket', values = 'sum_occ', aggfunc = 'sum')

df_pivot.reset_index(inplace = True)

df_pivot.head()

Bucket,Text,Screen,Tones,Trigger
0,CUSTOMER STATED THAT THE DEVICE ALARMED BUTTON...,1,0,0
1,DATE OF SUBMISSION 06/10/2011. DEVICE EVALUATI...,1,0,0
2,DATE OF SUBMISSION 08/21/2017 - DEVICE EVALUAT...,1,0,0
3,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",1,0,0
4,IT WAS REPORTED THAT THE DEVICE ALARMED AND IN...,1,1,0


In [77]:
# replicating multi-field

def multi_field (df):
    
   # iterating over columns

    for col in df.columns:
        
        # casting col as str
        
        df[col] = df[col].astype(str)
    
        # setting col header
        
        header = df[col].name
    
        # iterating over rows in col
        
        df[col] = df[col].apply(lambda x: header if x == '1' else pd.NA)
        
    return df

In [78]:
# applying function to all rows and just relevant columns

df_adjusted = multi_field(df_pivot.iloc[0:, 1:])

df_adjusted

Bucket,Screen,Tones,Trigger
0,Screen,<NA>,<NA>
1,Screen,<NA>,<NA>
2,Screen,<NA>,<NA>
3,Screen,<NA>,<NA>
4,Screen,Tones,<NA>
5,Screen,<NA>,<NA>
6,Screen,<NA>,<NA>
7,<NA>,<NA>,<NA>
8,<NA>,<NA>,Trigger
9,Screen,<NA>,<NA>


In [85]:
# concating text field with df

df_final = pd.concat([df_pivot['Text'], df_adjusted], axis = 1)

df_final.head()

,Text,Screen,Tones,Trigger
0,CUSTOMER STATED THAT THE DEVICE ALARMED BUTTON...,Screen,<NA>,<NA>
1,DATE OF SUBMISSION 06/10/2011. DEVICE EVALUATI...,Screen,<NA>,<NA>
2,DATE OF SUBMISSION 08/21/2017 - DEVICE EVALUAT...,Screen,<NA>,<NA>
3,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",Screen,<NA>,<NA>
4,IT WAS REPORTED THAT THE DEVICE ALARMED AND IN...,Screen,Tones,<NA>


In [87]:
# transposing

df_transposed = pd.melt(df_final, id_vars = ['Text'], value_vars = ['Screen', 'Tones', 'Trigger'] )

df_transposed.head()

,Text,variable,value
0,CUSTOMER STATED THAT THE DEVICE ALARMED BUTTON...,Screen,Screen
1,DATE OF SUBMISSION 06/10/2011. DEVICE EVALUATI...,Screen,Screen
2,DATE OF SUBMISSION 08/21/2017 - DEVICE EVALUAT...,Screen,Screen
3,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...",Screen,Screen
4,IT WAS REPORTED THAT THE DEVICE ALARMED AND IN...,Screen,Screen


In [104]:
# concatting value by text 

# creating function to handle nulls in value and concat

def concat (value):
    
    return ','.join([x for x in value if pd.notna(x)])
    

In [109]:
# applying function

df_sum = df_transposed.groupby('Text').agg(concat).reset_index()

df_sum.head()

,Text,variable,value
0,CUSTOMER STATED THAT THE DEVICE ALARMED BUTTON...,"Screen,Tones,Trigger",Screen
1,DATE OF SUBMISSION 06/10/2011. DEVICE EVALUATI...,"Screen,Tones,Trigger",Screen
2,DATE OF SUBMISSION 08/21/2017 - DEVICE EVALUAT...,"Screen,Tones,Trigger",Screen
3,"IT WAS REPORTED THAT DURING A PROCEDURE, IMMED...","Screen,Tones,Trigger",Screen
4,IT WAS REPORTED THAT THE DEVICE ALARMED AND IN...,"Screen,Tones,Trigger","Screen,Tones"


In [117]:
# counting number of occurence per value concat

s_count = df_sum['value'].value_counts()

# converting series into dataframe

df_count = pd.DataFrame(s_count)

# resetting index

df_count.reset_index(inplace = True)

df_count.head()

,value,count
0,Screen,7
1,"Screen,Tones",1
2,,1
3,Trigger,1


In [119]:
# filtering out null

df_count[df_count['value'] != '']

,value,count
0,Screen,7
1,"Screen,Tones",1
3,Trigger,1
